## 1. Setup and Imports

Import required libraries and set up the environment for statistics calculation.

## Environment Setup

Before running this notebook, make sure you have:

1. **Installed dependencies**: `pip install -r requirements.txt`
2. **Created environment file**: Copy `.env.example` to `.env` and update with your credentials:
   ```bash
   cp .env.example .env
   ```
3. **Updated credentials** in the `.env` file:
   ```
   API_BASE_URL=https://api.trends.earth
   USERNAME="your_email@example.com"
   PASSWORD="your_password"
   ```

In [7]:
import uuid
import warnings
from pathlib import Path
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns
from dotenv import load_dotenv
from trendsearth_api import TrendsEarthAPIClient

# Force reload environment variables from .env file
# This ensures we get the latest values even if already loaded
print("Loading environment variables...")

# Check if .env file exists
env_file = Path(".env")
if env_file.exists():
    print(f"Found .env file at: {env_file.absolute()}")
    # Force reload with override=True to refresh cached values
    load_dotenv(override=True)
    print("Environment variables reloaded")
else:
    print(f".env file not found at: {env_file.absolute()}")
    print("Please ensure the .env file is in the same directory as this notebook")

# Set style for visualizations
plt.style.use("default")
sns.set_palette("husl")

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

print("\nLibraries imported successfully.")
print("Environment variables loaded from .env file")

Loading environment variables...
Found .env file at: c:\Users\azvol\Code\LandDegradation\trends.earth\notebooks\.env
Environment variables reloaded

Libraries imported successfully.
Environment variables loaded from .env file


## 2. API Configuration and Authentication

Set up connection to the trends.earth API for statistics calculation.

In [5]:
# Initialize the API client
api_client = TrendsEarthAPIClient()

# Authenticate using environment variables from .env file
if api_client.authenticate_from_env():
    headers = api_client.get_auth_headers()
    print("Ready to submit statistics job")
else:
    print("❌ Failed to authenticate - check your .env file")
    print("Please ensure your .env file contains:")
    print("- API_USERNAME=your_username")
    print("- API_PASSWORD=your_password")
    print("- API_BASE_URL=https://api.trends.earth (optional)")
    headers = None

Using API URL from environment: https://api.trends.earth
Authenticating with Trends.Earth API...
   Using email: trends.earth-prais-server@trends.earth
Successfully authenticated with Trends.Earth API
Ready to submit statistics job


## 3. Configure Statistics Calculation Parameters

Set up the parameters for SDG 15.3.1 statistics calculation, including land degradation data and error polygons.

In [8]:
params = {
    "script_name": "sdg-15-3-1-summary",
    "iso": "KEN",  # Kenya ISO code
    "periods": ["baseline", "report_1", "report_2"],
    "crosstabs": [["baseline", "report_2"]],
    "boundary_dataset": "UN",
    "polygons": {
        "type": "FeatureCollection",
        "features": [
            {
                "type": "Feature",
                "properties": {
                    "id": 2,
                    "name": "Test Area 1 (Small)",
                    "uuid": str(uuid.uuid4()),
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [37.1, 0.1],
                            [37.12, 0.1],
                            [37.12, 0.12],
                            [37.1, 0.12],
                            [37.1, 0.1],
                        ]
                    ],
                },
            },
            {
                "type": "Feature",
                "properties": {
                    "id": 3,
                    "name": "Test Area 2 (Medium)",
                    "uuid": str(uuid.uuid4()),
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [37.2, 0.2],
                            [37.245, 0.2],
                            [37.245, 0.245],
                            [37.2, 0.245],
                            [37.2, 0.2],
                        ]
                    ],
                },
            },
            {
                "type": "Feature",
                "properties": {
                    "id": 4,
                    "name": "Test Area 3 (Large)",
                    "uuid": str(uuid.uuid4()),
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [
                        [
                            [37.3, 0.3],
                            [37.364, 0.3],
                            [37.364, 0.364],
                            [37.3, 0.364],
                            [37.3, 0.3],
                        ]
                    ],
                },
            },
        ],
    },
}

## 4. Submit Statistics Calculation Job

Submit the job to the trends.earth API and get the job ID for monitoring.

In [ ]:
if headers:
    print(" Submitting statistics calculation job...")
    execution_id = api_client.submit_job("sdg-15-3-1-stats-2-1-17", params)

    if execution_id:
        print(f"Job submitted successfully: {execution_id}")

        # Monitor the job until completion
        final_status = api_client.monitor_job(execution_id, max_minutes=15)

        if final_status:
            status = final_status.get("status", "unknown").upper()
            if status in ["SUCCESS", "FINISHED"]:
                print("✅ Statistics calculation completed successfully!")

                # The results should be available in the job data
                results = final_status.get("results", {})
                if results:
                    print("Results available")
                    pprint(results)
                else:
                    print("No results data found in job response")
            else:
                print(f"❌ Job failed with status: {status}")
        else:
            print("❌ Failed to monitor job")
    else:
        print("❌ Failed to submit statistics job")
else:
    print("❌ Cannot submit job: authentication failed")
    print("❌ Please check your credentials and try again")

 Submitting statistics calculation job...
✅ Job submitted successfully: 5206fde1-7ad3-4235-ad2a-5738b16a3257
Status: READY


## 5. Understanding the Results Format

The `results` object returned from the statistics calculation job is a `JsonResults` object with the following structure:

### Results Structure

```json
{
  "name": "sdg-15-3-1-statistics",
  "data": {
    "bands": {
      "baseline": {
        "name": "SDG 15.3.1 Indicator",
        "index": 1,
        "metadata": {}
      },
      "report_1": {
        "name": "SDG 15.3.1 Indicator (status)",
        "index": 2,
        "metadata": {}
      },
      "report_2": {
        "name": "SDG 15.3.1 Indicator (status)",
        "index": 3,
        "metadata": {}
      }
    },
    "stats": {
      "polygon_uuid_1": {
        "baseline": {
          "area_ha": 1234.5,
          "degraded_pct": 15.2,
          "stable_pct": 70.3,
          "improved_pct": 12.1,
          "nodata_pct": 2.4
        },
        "report_1": {
          "area_ha": 1234.5,
          "degraded_pct": 18.1,
          "stable_pct": 68.9,
          "improved_pct": 10.5,
          "nodata_pct": 2.5
        },
        "report_2": {
          "area_ha": 1234.5,
          "degraded_pct": 16.7,
          "stable_pct": 69.8,
          "improved_pct": 11.2,
          "nodata_pct": 2.3
        },
        "crosstabs": [
          {
            "band_1": "baseline",
            "band_2": "report_2",
            "total_area_ha": 1234.5,
            "crosstab": {
              "degraded": {
                "degraded": {"area_ha": 123.4, "area_pct": 10.0},
                "stable": {"area_ha": 45.6, "area_pct": 3.7},
                "improved": {"area_ha": 18.9, "area_pct": 1.5}
              },
              "stable": {
                "degraded": {"area_ha": 67.8, "area_pct": 5.5},
                "stable": {"area_ha": 789.1, "area_pct": 63.9},
                "improved": {"area_ha": 89.2, "area_pct": 7.2}
              },
              "improved": {
                "degraded": {"area_ha": 12.3, "area_pct": 1.0},
                "stable": {"area_ha": 34.5, "area_pct": 2.8},
                "improved": {"area_ha": 53.7, "area_pct": 4.4}
              }
            },
            "band_1_totals": {
              "degraded": {"area_ha": 187.9, "area_pct": 15.2},
              "stable": {"area_ha": 946.1, "area_pct": 76.6},
              "improved": {"area_ha": 100.5, "area_pct": 8.1}
            },
            "band_2_totals": {
              "degraded": {"area_ha": 203.5, "area_pct": 16.5},
              "stable": {"area_ha": 868.2, "area_pct": 70.3},
              "improved": {"area_ha": 162.8, "area_pct": 13.2}
            }
          }
        ]
      },
      "polygon_uuid_2": {
        "...": "similar structure for each polygon"
      }
    }
  },
  "type": "JsonResults"
}
```

### Key Components Explained

**Top Level:**
- `name`: Always "sdg-15-3-1-statistics" 
- `data`: Contains the actual statistics data
- `type`: Always "JsonResults"

**Band Information (`data.bands`):**
- Keys are period names: "baseline", "report_1", "report_2"
- Each band has `name`, `index` (1-based raster band number), and `metadata`

**Statistics (`data.stats`):**
- Keys are polygon UUIDs from your input polygons
- Each polygon contains statistics for each band plus optional crosstabs

**Per-Band Statistics:**
- `area_ha`: Total area of the polygon in hectares
- `degraded_pct`: Percentage of area classified as degraded
- `stable_pct`: Percentage of area classified as stable  
- `improved_pct`: Percentage of area classified as improved
- `nodata_pct`: Percentage of area with no data

**Crosstab Analysis (when requested):**
- `band_1`: Reference to first band (period name)
- `band_2`: Reference to second band (period name) 
- `total_area_ha`: Total analysis area
- `crosstab`: 3x3 matrix showing transitions from first band to second band
- `band_1_totals`: Summary totals for the first band
- `band_2_totals`: Summary totals for the second band
- Each cell shows both area (`area_ha`) and percentage (`area_pct`)

### Notes

- **All areas in hectares**: Area measurements are always in hectares
- **Percentages**: All percentages are relative to the total polygon area
- **Crosstabs include totals**: Both row totals (`band_1_totals`) and column totals (`band_2_totals`)
- **Crosstabs are optional**: Only included if you specify `crosstabs` parameter
- **One polygon per UUID**: Each input polygon gets its own statistics section